In [1]:
from Solid.StochasticHillClimb import StochasticHillClimb
from sklearn.metrics import mean_squared_error
from tengp.individual import IndividualBuilder, NPIndividual
from tengp import Parameters, FunctionSet
import numpy as np

In [39]:
class Algorithm(StochasticHillClimb):
    """
    Optimize the CCGP individual
    """
    
    def _neighbor(self):
        new_genes = np.array(self.current_state.genes[:])
        bounds = self.current_state.bounds
        params = self.current_state.params
        
        chances = np.random.random(len(new_genes))
        mask = chances < 0.1
        new_genes[mask] = np.random.normal(new_genes[mask], 2)
        new_genes = np.clip(new_genes, bounds[0], bounds[1])
        
        new_individual = NPIndividual(new_genes.tolist(), bounds, params)
        output = new_individual.transform(x_train)
        try:
            fitness = -mean_squared_error(output, y_train)
        except ValueError:
            fitness = -10e10
        return new_individual
    
    def _objective(self, member):
        if member.fitness is not None:
            return member.fitness
        
        output = member.transform(x_train)
        try:
            fitness = -mean_squared_error(output, y_train)
        except ValueError:
            fitness = -10e10
        
        member.fitness = fitness
        
        return fitness

In [41]:
# define CCGP system
from experiment_settings import nguyen7_funset
import symreg
import random

params = Parameters(2, 1, 1, 100, nguyen7_funset, real_valued=True, max_back=10)

ib = IndividualBuilder(params)

In [42]:
x_train, y_train, _, _ = symreg.get_benchmark_nguyen7(random, None)
x_train = np.c_[np.ones(len(x_train)), x_train]

In [45]:
initial_individual = ib.create()

output = initial_individual.transform(x_train)
initial_individual.fitness = -mean_squared_error(output, y_train)

algo = Algorithm(initial_individual, 1, 100000, 0)

In [46]:
%%time

algo.run(verbose=False)

/home/jarino/anaconda3/envs/tengp/lib/python3.7/site-packages/sklearn/metrics/regression.py:241: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0,
/home/jarino/anaconda3/envs/tengp/lib/python3.7/site-packages/sklearn/metrics/regression.py:241: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0,


KeyboardInterrupt: 